#### 0. LangChain에서 도구(tool) 활용 방법

In [1]:
from dotenv import load_dotenv
import os
# .env 파일을 불러와서 환경 변수로 설정
load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
print(OPENAI_API_KEY[:2])

UPSTAGE_API_KEY = os.getenv("UPSTAGE_API_KEY")
print(UPSTAGE_API_KEY[30:])

sk
Fy


In [2]:
# from langchain_openai import ChatOpenAI

# llm = ChatOpenAI(
#     base_url="https://api.groq.com/openai/v1",
#     #model="meta-llama/llama-4-scout-17b-16e-instruct",
#     model="moonshotai/kimi-k2-instruct-0905",
#     temperature=0
# )

from langchain_upstage import ChatUpstage
llm = ChatUpstage(
        model="solar-pro",
        base_url="https://api.upstage.ai/v1",
        temperature=0.5
    )
print(llm)

c:\Users\user\AppData\Local\pypoetry\Cache\virtualenvs\mylangchain-app-SBe-Yh6W-py3.12\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


client=<openai.resources.chat.completions.completions.Completions object at 0x000001255A1CB2C0> async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x000001255A1F5700> model_name='solar-pro' temperature=0.5 model_kwargs={} upstage_api_key=SecretStr('**********') upstage_api_base='https://api.upstage.ai/v1'


- [tool decorator](https://python.langchain.com/docs/how_to/custom_tools/#tool-decorator)를 사용하면 쉽게 도구를 만들 수 있습니다

In [3]:
from langchain_core.tools import tool

@tool
def add(a: int, b: int) -> int:
    """숫자 a와 b를 더합니다."""
    return a + b

@tool
def multiply(a: int, b: int) -> int:
    """숫자 a와 b를 곱합니다."""
    return a * b

- LLM을 호출했을 때와 도구를 사용했을 때의 차이를 알아봅니다

In [7]:
query = '3 곱하기 5는?'
llm_result = llm.invoke(query)

print(llm_result.content)

3 곱하기 5는 **15**입니다.  

계산 과정:  
3 × 5 = 15  

간단히 설명하면, 3을 5번 더한 것과 같습니다.  
(3 + 3 + 3 + 3 + 3 = 15)  

도움이 되었기를 바랍니다! 😊


- 도구 리스트는 LLM에 해당하는 `BaseModel` 클래스에 `bind_tools` 메서드를 통해 전달합니다

In [57]:
llm_with_tools = llm.bind_tools([add, multiply])

print(type(llm_with_tools))
print(llm_with_tools)

<class 'langchain_core.runnables.base.RunnableBinding'>
bound=ChatUpstage(client=<openai.resources.chat.completions.completions.Completions object at 0x00000142DE786D50>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x00000142DE795940>, model_name='solar-pro', temperature=0.5, model_kwargs={}, upstage_api_key=SecretStr('**********'), upstage_api_base='https://api.upstage.ai/v1') kwargs={'tools': [{'type': 'function', 'function': {'name': 'add', 'description': '숫자 a와 b를 더합니다.', 'parameters': {'properties': {'a': {'type': 'integer'}, 'b': {'type': 'integer'}}, 'required': ['a', 'b'], 'type': 'object'}}}, {'type': 'function', 'function': {'name': 'multiply', 'description': '숫자 a와 b를 곱합니다.', 'parameters': {'properties': {'a': {'type': 'integer'}, 'b': {'type': 'integer'}}, 'required': ['a', 'b'], 'type': 'object'}}}]} config={} config_factories=[]


- `AIMessage`의 `additional_kwargs` 속성은 `tool_calls`를 포함합니다
- `tool_calls`는 도구를 호출하는 메시지를 포함합니다

In [58]:
tool_result = llm_with_tools.invoke(query)

tool_result

AIMessage(content='[The question directly asks for the product of 3 and 5, which is ESSENTIAL to answer using the `multiply` function. No other operations or functions are necessary.]  \n\nResult: 15', additional_kwargs={'tool_calls': [{'id': 'chatcmpl-tool-a9a7ac780be9491499d39eef7aa88752', 'function': {'arguments': '{"a": 3, "b": 5}', 'name': 'multiply'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 66, 'prompt_tokens': 578, 'total_tokens': 644, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'solar-pro2-250909', 'system_fingerprint': None, 'id': '14ee0f64-1c8b-4060-b6de-338c3723d48c', 'service_tier': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--ee7a8701-7b15-4977-a3c0-958ce6d8aa75-0', tool_calls=[{'name': 'multiply', 'args': {'a': 3, 'b': 5}, '

In [59]:
tool_result.tool_calls

[{'name': 'multiply',
  'args': {'a': 3, 'b': 5},
  'id': 'chatcmpl-tool-a9a7ac780be9491499d39eef7aa88752',
  'type': 'tool_call'}]

In [44]:
from typing import Sequence

from langchain_core.messages import AnyMessage, HumanMessage

human_message = HumanMessage(query)
message_list: Sequence[AnyMessage] = [human_message] 


- `tool_calls` 속성은 도구를 호출하는 메시지를 포함합니다
- `tool_calls`를 가진 `AIMessage`의 형태를 기억하기

In [45]:
ai_message = llm_with_tools.invoke(message_list)
ai_message

AIMessage(content='[주어진 질문은 "3 곱하기 5"의 결과를 묻는 것으로, 이는 직접적으로 곱셈 연산이 필요합니다. 따라서 \'multiply\' 함수를 호출하는 것이 필수적입니다. 다른 방법으로는 일반 지식으로도 답할 수 있지만, 규칙에 따라 함수 사용이 필요한 경우에만 호출해야 하며, 이 경우 곱셈 연산이 질문의 핵심이므로 함수 호출이 정당합니다.] \n\n정답: 15', additional_kwargs={'tool_calls': [{'id': 'chatcmpl-tool-638066a86a0143c682396fe4fde379e5', 'function': {'arguments': '{"a": 3, "b": 5}', 'name': 'multiply'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 89, 'prompt_tokens': 578, 'total_tokens': 667, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'solar-pro2-250909', 'system_fingerprint': None, 'id': '11c36d9a-83e6-406f-b0a9-0ee6e9525b33', 'service_tier': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--1e6c9b67-caf5-4549-aac5-abddf1b12069-0', tool_calls=[{'name': 'multiply', 'args': 

In [46]:
ai_message.tool_calls

[{'name': 'multiply',
  'args': {'a': 3, 'b': 5},
  'id': 'chatcmpl-tool-638066a86a0143c682396fe4fde379e5',
  'type': 'tool_call'}]

In [47]:
from pprint import pprint

message_list.append(ai_message)

pprint(message_list)

[HumanMessage(content='3 곱하기 5는?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='[주어진 질문은 "3 곱하기 5"의 결과를 묻는 것으로, 이는 직접적으로 곱셈 연산이 필요합니다. 따라서 \'multiply\' 함수를 호출하는 것이 필수적입니다. 다른 방법으로는 일반 지식으로도 답할 수 있지만, 규칙에 따라 함수 사용이 필요한 경우에만 호출해야 하며, 이 경우 곱셈 연산이 질문의 핵심이므로 함수 호출이 정당합니다.] \n\n정답: 15', additional_kwargs={'tool_calls': [{'id': 'chatcmpl-tool-638066a86a0143c682396fe4fde379e5', 'function': {'arguments': '{"a": 3, "b": 5}', 'name': 'multiply'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 89, 'prompt_tokens': 578, 'total_tokens': 667, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'solar-pro2-250909', 'system_fingerprint': None, 'id': '11c36d9a-83e6-406f-b0a9-0ee6e9525b33', 'service_tier': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--1

- `AIMessage`의 `tool_calls`를 활용해서 도구를 직접 호출할 수도 있습니다

In [48]:
ai_message.tool_calls[0]

{'name': 'multiply',
 'args': {'a': 3, 'b': 5},
 'id': 'chatcmpl-tool-638066a86a0143c682396fe4fde379e5',
 'type': 'tool_call'}

In [49]:
tool_message = multiply.invoke(ai_message.tool_calls[0])

print(tool_message)

content='15' name='multiply' tool_call_id='chatcmpl-tool-638066a86a0143c682396fe4fde379e5'


- 하지만 에이전트의 경우 도구를 직접 호출하는 것이 아니라 도구를 호출하는 메시지를 만들어서 전달합니다

In [50]:
message_list.append(tool_message)

pprint(message_list)

[HumanMessage(content='3 곱하기 5는?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='[주어진 질문은 "3 곱하기 5"의 결과를 묻는 것으로, 이는 직접적으로 곱셈 연산이 필요합니다. 따라서 \'multiply\' 함수를 호출하는 것이 필수적입니다. 다른 방법으로는 일반 지식으로도 답할 수 있지만, 규칙에 따라 함수 사용이 필요한 경우에만 호출해야 하며, 이 경우 곱셈 연산이 질문의 핵심이므로 함수 호출이 정당합니다.] \n\n정답: 15', additional_kwargs={'tool_calls': [{'id': 'chatcmpl-tool-638066a86a0143c682396fe4fde379e5', 'function': {'arguments': '{"a": 3, "b": 5}', 'name': 'multiply'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 89, 'prompt_tokens': 578, 'total_tokens': 667, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'solar-pro2-250909', 'system_fingerprint': None, 'id': '11c36d9a-83e6-406f-b0a9-0ee6e9525b33', 'service_tier': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--1

In [51]:
tool_result = llm_with_tools.invoke(message_list)

pprint(tool_result)

AIMessage(content='3 곱하기 5는 15입니다. \n\n함수 호출 결과와 일치하는 정답을 확인했습니다.\n\n[질문의 핵심인 "3 곱하기 5" 연산을 수행하기 위해 \'multiply\' 함수가 필수적으로 요구됩니다. 일반 지식으로도 답변 가능하지만, 문제 해결을 위해 함수 사용이 명시적으로 요청된 상황으로 판단됩니다.] \n\n최종 답변: 15', additional_kwargs={'tool_calls': [{'id': 'chatcmpl-tool-99c188eebb814a838f6526d0c3606084', 'function': {'arguments': '{"a": 3, "b": 5}', 'name': 'multiply'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 86, 'prompt_tokens': 682, 'total_tokens': 768, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'solar-pro2-250909', 'system_fingerprint': None, 'id': '37df693b-c96e-440c-8257-b928d4e05a01', 'service_tier': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--bb7bb29f-23f3-4d89-aecd-5884b8a4ccb9-0', tool_calls=[{'name': 'multiply', 'args': {'a': 3, 'b': 5}

- `message_list`의 순서를 기억하기

In [52]:
message_list

[HumanMessage(content='3 곱하기 5는?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='[주어진 질문은 "3 곱하기 5"의 결과를 묻는 것으로, 이는 직접적으로 곱셈 연산이 필요합니다. 따라서 \'multiply\' 함수를 호출하는 것이 필수적입니다. 다른 방법으로는 일반 지식으로도 답할 수 있지만, 규칙에 따라 함수 사용이 필요한 경우에만 호출해야 하며, 이 경우 곱셈 연산이 질문의 핵심이므로 함수 호출이 정당합니다.] \n\n정답: 15', additional_kwargs={'tool_calls': [{'id': 'chatcmpl-tool-638066a86a0143c682396fe4fde379e5', 'function': {'arguments': '{"a": 3, "b": 5}', 'name': 'multiply'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 89, 'prompt_tokens': 578, 'total_tokens': 667, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'solar-pro2-250909', 'system_fingerprint': None, 'id': '11c36d9a-83e6-406f-b0a9-0ee6e9525b33', 'service_tier': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--1